In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

# Imports and feature selection

In [96]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')

In [97]:
# drop
docs.drop(['Middle Name','Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [98]:
indem = claims[['Indemnity_Paid','ProviderId']]

In [99]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [100]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

# Join docs and indemnity tables

In [101]:
joined = pd.merge(docs, indem, how='outer', on='ProviderId')
joined.drop_duplicates(inplace=True)

# Claims data
#### building classifiers for BoardCredit and ClaimsBand, then merging with joined

In [103]:
# resave debs_and_creds with just the desired columns
debs_and_creds = debs_and_creds[['ProviderId','BoardCredit','ClaimsBand']]

# lower Provider Id
# reset index required
debs_and_creds['ProviderId'] = debs_and_creds['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())
debs_and_creds.tail()

,ProviderId,BoardCredit,ClaimsBand
8458,19d9c4e6-e820-41c4-b398-134e6c0ba410,-55,-220
8459,6398eeea-10af-448f-bd10-364be1d0c90c,0,-8951
8460,d54c0b59-08e5-4ed7-ba5a-413b2016b269,-1366,0
8461,3eacedf6-9173-4a64-89c7-ba8b6e150850,-88,0
8462,ea83f592-67dd-46d5-b15c-9d0738f81c93,0,-258


In [104]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()
debs_and_creds.reset_index(inplace=True)

In [105]:
def build_classifier(item):
    if item == 0:
        return 0
    if item != 0:
        return 1

In [106]:
# use build_classifier to create binary column from BoardCredit & ClaimsBand
debs_and_creds['board_cert'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['claims_band'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))

In [107]:
joined = pd.merge(joined, debs_and_creds[['ProviderId','board_cert', 'claims_band']], how='outer', on='ProviderId')

# EDA

In [111]:
joined[joined['Indemnity_Paid'] > 0].sort(['Last Name'])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,ProviderId,Last Name,First Name,Gender,Date of Birth,City,State,County,Indemnity_Paid,board_cert,claims_band
78,1c50184d-410a-4c07-b6a0-48b27c0456da,Akins,James,Male,11/7/66,Scott Depot,WV,PUTNAM,199999.99,0.0,0.0
97,1cf4827a-fc9b-410a-8923-c2b19e9f8905,Alexander,Ian,Male,9/11/74,Los Alamos,NM,LOS ALAMOS,125000.00,1.0,0.0
135,fef26456-6759-4573-a69d-70482e29cea1,Alva,Brick,Male,10/13/70,Houston,TX,HARRIS,90000.00,1.0,1.0
379,80f8200e-12a4-4bec-b531-22b4f4fc2e4e,Ancillary,MidLevel,Male,NaN,Marlin,TX,FALLS,22000.00,NaN,NaN
2025,53294bf4-9f2d-4e7e-b44e-e34e7ac89cbd,Balder,Donald,Male,8/1/63,Covington,LA,HARRISON,850000.00,1.0,1.0
2057,7590e9fa-b86c-40eb-add1-5cab8cf7c98d,Barnes,Bryan,Male,4/5/70,Joplin,MO,NEWTON,180000.00,1.0,1.0
2058,7590e9fa-b86c-40eb-add1-5cab8cf7c98d,Barnes,Bryan,Male,4/5/70,Joplin,MO,NEWTON,195000.00,1.0,1.0
2198,3bbdfc67-dccc-4a69-81c0-8d98fdd6ddca,Blount,Robin,Unknown,10/22/57,Columbia,MO,BOONE,150000.00,1.0,1.0
2245,4aad012f-421b-4324-9fb1-ab95b2930c48,Boveri,Stephen,Unknown,1/29/63,Fenton,MO,SAINT LOUIS,395000.00,1.0,1.0
2255,d06b0d68-8cab-4e90-9f37-8b9751b9d1aa,Bowne,David,Male,7/25/63,nan,nan,nan,675000.00,1.0,1.0


# Feature engineering

In [10]:
# build age column
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [11]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [12]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [13]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [14]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [16]:
def to_years(dt):
    if dt:
        dt = int(dt.strftime("%y"))
    return dt

In [17]:
# TODO: convert days to years in joined['age']
# TODO: drop 1/1/2060 instances
joined['age'] = joined['age'].apply(lambda dt: to_years(dt))

AttributeError: 'Timedelta' object has no attribute 'strftime'